In [1]:
import sys
print(sys.path)
sys.path.append("../KE/")
print(sys.path)

['/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe/notebooks', '/Users/marcin/.pyenv/versions/3.10.11/lib/python310.zip', '/Users/marcin/.pyenv/versions/3.10.11/lib/python3.10', '/Users/marcin/.pyenv/versions/3.10.11/lib/python3.10/lib-dynload', '', '/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe/notebooks/venv/lib/python3.10/site-packages']
['/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe/notebooks', '/Users/marcin/.pyenv/versions/3.10.11/lib/python310.zip', '/Users/marcin/.pyenv/versions/3.10.11/lib/python3.10', '/Users/marcin/.pyenv/versions/3.10.11/lib/python3.10/lib-dynload', '', '/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe/notebooks/venv/lib/python3.10/site-packages', '../KE/']


In [5]:
!pip install imblearn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 8.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, cv

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

from KE.classifier.prepare import split_dataset
from KE.dataclean import tables

In [7]:
data = pd.read_csv("../data/merged/merged-kg_edges.tsv", header=0, sep="\t")

/var/folders/7w/ksxs106s7_sg14v6cb2y70vr0000gn/T/ipykernel_47548/2785372968.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/merged/merged-kg_edges.tsv", header=0, sep="\t")


In [8]:
data

,id,subject,predicate,object,relation,provided_by,knowledge_source,meta,primary_knowledge_source
0,urn:uuid:b6cd7c15-230e-43d1-ba8a-b2a8b1733cbe,NCBITaxon:1,biolink:subclass_of,BFO:0000040,rdfs:subClassOf,NaN,Graph,NaN,NaN
1,urn:uuid:88f29617-6c27-4b5d-8826-8e43248ddad9,NCBITaxon:10,biolink:subclass_of,NCBITaxon:1706371,rdfs:subClassOf,NaN,Graph,NaN,NaN
2,urn:uuid:43701742-c66a-4b28-87c5-236add5ab405,NCBITaxon:100,biolink:subclass_of,NCBITaxon:99,rdfs:subClassOf,NaN,Graph,NaN,NaN
3,urn:uuid:245e5bc2-090d-43af-ac7d-843903a9dd52,NCBITaxon:100,biolink:occurs_in,mediadive.medium:7,BAO:0002924,NaN,Graph,NaN,bacdive:17385
4,urn:uuid:50073882-f09b-418d-bb94-5832b7ce1af4,NCBITaxon:100,biolink:capable_of,traits.pathways:aerobic_chemo_heterotrophy,RO:0002215,NaN,Graph,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1469025,urn:uuid:1b21e8fd-6c66-4045-a252-026bac0e9e9d,NCBITaxon:147448,biolink:has_phenotype,traits.cell_shape_enum:coccus,RO:0002200,NaN,Graph,NaN,NaN
1469026,urn:uuid:567ad208-c607-40bd-8ec6-d2e73d753c81,NCBITaxon:147448,biolink:capable_of,ECOCORE:00000177,RO:0002215,NaN,Graph,NaN,NaN
1469027,urn:uuid:6981ca41-abda-4872-a5fe-1db001032db7,NCBITaxon:1240726,biolink:has_phenotype,traits.cell_shape_enum:bacillus,RO:0002200,NaN,Graph,NaN,NaN
1469028,urn:uuid:0a2c871e-5678-4fff-b2da-f9e22e709f36,NCBITaxon:1240783,biolink:has_phenotype,traits.cell_shape_enum:bacillus,RO:0002200,NaN,Graph,NaN,NaN


In [9]:
data_pairs = data[['subject','object']].drop_duplicates()
data_pairs

,subject,object
0,NCBITaxon:1,BFO:0000040
1,NCBITaxon:10,NCBITaxon:1706371
2,NCBITaxon:100,NCBITaxon:99
3,NCBITaxon:100,mediadive.medium:7
4,NCBITaxon:100,traits.pathways:aerobic_chemo_heterotrophy
...,...,...
1469025,NCBITaxon:147448,traits.cell_shape_enum:coccus
1469026,NCBITaxon:147448,ECOCORE:00000177
1469027,NCBITaxon:1240726,traits.cell_shape_enum:bacillus
1469028,NCBITaxon:1240783,traits.cell_shape_enum:bacillus


In [12]:
# Subset the DataFrame based on the substring in subject
data_pairs_clean = data_pairs[data_pairs['subject'].str.contains('NCBITaxon:')]
# Subset the DataFrame based on the substring in object
data_pairs_clean = data_pairs_clean[data_pairs_clean['object'].str.contains('mediadive.medium:')]

In [13]:
#data_pairs_clean = tables.rows_and_cols_quant_filter(data_pairs)
#require at least 10 examples per class
data_pairs_clean = data_pairs_clean.groupby('object').filter(lambda x : len(x)>9)

In [14]:
data_pairs_clean

,subject,object
148,NCBITaxon:100053,mediadive.medium:1113
196,NCBITaxon:1000566,mediadive.medium:1504
437,NCBITaxon:1001240,mediadive.medium:92
915,NCBITaxon:1002227,mediadive.medium:1122
1897,NCBITaxon:1004279,mediadive.medium:98
...,...,...
1374878,NCBITaxon:771855,mediadive.medium:1
1374880,NCBITaxon:196180,mediadive.medium:220
1375025,NCBITaxon:2512148,mediadive.medium:1504
1376898,NCBITaxon:67334,mediadive.medium:65


In [15]:
# Splitting the data into features and target labels
X = data_pairs_clean[['subject']]
y = data_pairs_clean['object']

# Convert the input labels column to a categorical type if it isn't
X['subject'] = X['subject'].astype('category')

# Convert categorical columns to integers
X['subject'] = X['subject'].cat.codes

/var/folders/7w/ksxs106s7_sg14v6cb2y70vr0000gn/T/ipykernel_47548/29537296.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['subject'] = X['subject'].astype('category')
/var/folders/7w/ksxs106s7_sg14v6cb2y70vr0000gn/T/ipykernel_47548/29537296.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['subject'] = X['subject'].cat.codes


In [16]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train the model
train_data = Pool(data=X_train, label=y_train, cat_features=[0])
test_data = Pool(data=X_test, label=y_test, cat_features=[0])


In [17]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, verbose=100)
model.fit(train_data)#, plot=True)

0:	learn: 4.1428631	total: 343ms	remaining: 5m 43s
100:	learn: 3.3925584	total: 36s	remaining: 5m 20s
200:	learn: 3.3179906	total: 1m 14s	remaining: 4m 55s
300:	learn: 3.2834237	total: 1m 52s	remaining: 4m 20s
400:	learn: 3.2623700	total: 2m 30s	remaining: 3m 45s
500:	learn: 3.2488218	total: 3m 8s	remaining: 3m 7s
600:	learn: 3.2388298	total: 3m 46s	remaining: 2m 30s
700:	learn: 3.2316522	total: 4m 24s	remaining: 1m 52s
800:	learn: 3.2266787	total: 5m 2s	remaining: 1m 15s
900:	learn: 3.2225698	total: 5m 40s	remaining: 37.4s
999:	learn: 3.2194485	total: 6m 18s	remaining: 0us


In [18]:
# Predict on test data
y_pred = model.predict(test_data)
y_pred_proba = model.predict_proba(test_data)[:,1]  # Probabilities for the positive class

# Print metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_pred_proba))

Accuracy: 0.17868098159509202

Classification Report:
                         precision    recall  f1-score   support

    mediadive.medium:1       0.20      0.12      0.15        73
  mediadive.medium:104       0.00      0.00      0.00        27
 mediadive.medium:1040       0.00      0.00      0.00         2
 mediadive.medium:104b       0.00      0.00      0.00        16
 mediadive.medium:104c       0.00      0.00      0.00         4
mediadive.medium:1076b       0.00      0.00      0.00        12
   mediadive.medium:11       0.00      0.00      0.00        27
  mediadive.medium:110       0.00      0.00      0.00        16
 mediadive.medium:110a       0.00      0.00      0.00         2
 mediadive.medium:1113       0.00      0.00      0.00         2
 mediadive.medium:1122       0.00      0.00      0.00         4
 mediadive.medium:1166       0.00      0.00      0.00         6
  mediadive.medium:119       0.00      0.00      0.00         9
 mediadive.medium:1193       0.00      0.00     

/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe/notebooks/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe/notebooks/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe/notebooks/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 

ValueError: multi_class must be in ('ovo', 'ovr')

In [19]:
import requests
from xml.etree import ElementTree
import pandas as pd

def get_taxon_genome_features(taxon_id):
    # Base URL for E-utilities
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
    
    # Use esearch to search for the taxon identifier in the taxonomy database
    search_url = f"{base_url}esearch.fcgi?db=taxonomy&term={taxon_id}&retmode=json"
    search_response = requests.get(search_url)
    search_data = search_response.json()
    
    # Obtain the Taxonomy ID from search response
    tax_id = search_data["esearchresult"]["idlist"][0]

    # Use efetch to get genome features using the taxonomy ID
    fetch_url = f"{base_url}efetch.fcgi?db=genome&id={tax_id}&rettype=fasta&retmode=xml"
    fetch_response = requests.get(fetch_url)
    genome_data = ElementTree.fromstring(fetch_response.content)
    
    # Parse the XML to extract genome features
    features = {}
    for feature in genome_data.findall('.//Feature'):
        feature_key = feature.find('Feature_key').text
        feature_value = feature.find('Feature_value').text
        if feature_key and feature_value:
            features[feature_key] = features.get(feature_key, []) + [feature_value]
    
    return features

def save_to_tsv(data, output_file):
    # Convert the dictionary to a pandas DataFrame
    df = pd.DataFrame.from_dict(data, orient='index')
    
    # Transpose the DataFrame so that taxon identifiers are row names
    df = df.transpose()
    
    # Save the DataFrame as TSV
    df.to_csv(output_file, sep='\t', header=True, index=True)

# Example Taxon ID
taxon_id = "NCBITaxon:100053"

# Strip 'NCBITaxon:' prefix to use with NCBI API
clean_taxon_id = taxon_id.replace("NCBITaxon:", "")

# Obtain genome features
genome_features = get_taxon_genome_features(clean_taxon_id)

# Specify the output file name
output_tsv = "genome_features.tsv"

# Save the genome features to a TSV file
save_to_tsv(genome_features, output_tsv)

print(f"Genome features saved to {output_tsv}")

IndexError: list index out of range

In [24]:
import requests

def get_uniprot_proteome_id(ncbi_taxon_id):
    # Remove the prefix 'NCBITaxon:' if present
    ncbi_taxon_id = ncbi_taxon_id.replace('NCBITaxon:', '')

    # UniProt search URL
    url = 'https://rest.uniprot.org/proteomes/search'
    params = {
        'query': f'taxonomy:{ncbi_taxon_id}',
        'format': 'list'
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        # Parse the response text for proteome IDs
        proteome_ids = response.text.strip().split('\n')
        if proteome_ids:
            return proteome_ids  # Return a list of found proteome IDs
        else:
            print(f"No proteome found for NCBI Taxon ID: {ncbi_taxon_id}")
            return None
    else:
        print(f"Failed to retrieve data from UniProt for NCBI Taxon ID: {ncbi_taxon_id}")
        return None

# Example usage
ncbi_taxon_id = 'NCBITaxon:100053'
clean_taxon_id = ncbi_taxon_id.replace("NCBITaxon:", "")
proteome_ids = get_uniprot_proteome_id(clean_taxon_id)

if proteome_ids:
    for proteome_id in proteome_ids:
        print(f"Found UniProtKB proteome identifier: {proteome_id}")


Failed to retrieve data from UniProt for NCBI Taxon ID: 100053


In [25]:
for taxon in data_pairs_clean.subject:
    taxon = taxon.replace("NCBITaxon:", "")
    proteome_ids = get_uniprot_proteome_id(taxon)

    if proteome_ids:
        for proteome_id in proteome_ids:
            print(f"Found UniProtKB proteome identifier: {proteome_id}")

Failed to retrieve data from UniProt for NCBI Taxon ID: 100053
Failed to retrieve data from UniProt for NCBI Taxon ID: 1000566
Failed to retrieve data from UniProt for NCBI Taxon ID: 1001240
Failed to retrieve data from UniProt for NCBI Taxon ID: 1002227
Failed to retrieve data from UniProt for NCBI Taxon ID: 1004279
Failed to retrieve data from UniProt for NCBI Taxon ID: 1004304
Failed to retrieve data from UniProt for NCBI Taxon ID: 1004316
Failed to retrieve data from UniProt for NCBI Taxon ID: 1004322
Failed to retrieve data from UniProt for NCBI Taxon ID: 1004326
Failed to retrieve data from UniProt for NCBI Taxon ID: 1004583
Failed to retrieve data from UniProt for NCBI Taxon ID: 1005
Failed to retrieve data from UniProt for NCBI Taxon ID: 1005740
Failed to retrieve data from UniProt for NCBI Taxon ID: 1005928
Failed to retrieve data from UniProt for NCBI Taxon ID: 1005944
Failed to retrieve data from UniProt for NCBI Taxon ID: 1005945
Failed to retrieve data from UniProt for NCB

KeyboardInterrupt: 